# Gaia Data Workshop - Heidelberg, November 21-24, 2016 
## The Gaia Service at AIP
Gal Matijevic // gmatijevic@aip.de
## Hands-on Tutorial

This notebook will cover the access of the AIP's Gaia service through the UWS (Universal Worker Service) interface. More information about the UWS standard can be found <a href="http://www.ivoa.net/documents/UWS/">here</a>. Another two very useful sources (in pdf format) on the topic are available from <a href="http://www.g-vo.org/tutorials/uwsintro.pdf">here</a> and <a href="http://wiki.ivoa.net/internal/IVOA/InterOpMay2016-GWS/uws-client.pdf">here</a>.

In this tutorial we will be using the <a href='https://github.com/aipescience/uws-client'>`uws-client`</a> for python.

### Basics

First, let us import the packages we will need in this tutorial:

In [ ]:
import time

import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

import uws.UWS.client as client

The connection to the database is established very easly through the `Client` object. We need to supply it the url and the user credentials (same as the ones used in the web interface):

In [ ]:
url = 'https://gaia.aip.de/uws/query'
username = ''
password = ''

In [ ]:
cli = client.Client(url, username, password)

To list all `PENDING` or `COMPLETED` jobs we can use the `get_job_list()` function (it might take a second or two):

In [ ]:
filters = {'phases': ['PENDING', 'COMPLETED']}
job_list = cli.get_job_list(filters)
for ref in job_list.job_reference:
    print ref.ownerId, ref.creationTime, ref.phase[0]

Similar job list can also be shown for other `phases` such as `ABORTED`, `QUEUED` and so on. Jobs can also be listed based on the time of their creation time or their consequtive number.

In [ ]:
filters = {'last': 2}
job_list = cli.get_job_list(filters)
for ref in job_list.job_reference:
    print ref

### Controling jobs

Adding a new job to the stack is done with the `new_job()` function. It requires a query and a queue to be passed to it. We wrap both into a dictionary called `parameters`:

In [ ]:
parameters = {'query': 'SELECT ra, `dec` FROM GDR1.tgas_source LIMIT 10',
              'queue': 'long'}
job = cli.new_job(parameters)
print job.phase

We can now run it with `run_job()` function. To check the phase of the job we use the `get_job()` to query its state every 10 seconds and see if the phase has changed from `QUEUED`.

In [ ]:
run = cli.run_job(job.job_id)
job = cli.get_job(run.job_id, wait='10', phase='QUEUED')
print job.phase[0]

If it is still `EXECUTING` we can re-check the phase with

In [ ]:
print job.phase[0]

If we look at the job list in the web interface we will see the submitted job in the list on the left.

Now we need to fetch the results that the query has generated. We can download the data returned by the query in a few different formats. We will be using the `csv` format as it is easly parsed by the `pandas` package that we will use to read the data into the notebook:

In [ ]:
if job.phase[0] == 'COMPLETED':
    fileurl = str(job.results[0].reference)
    cli.connection.download_file(fileurl, username, password,
                                 file_name='res.csv')

In [ ]:
data = pd.read_csv('res.csv')

In [ ]:
data

Finally, we can delete the job from the stack so it does not hog our limited user space. We do that using the `delete_job()` function:

In [ ]:
deleted = cli.delete_job(job.job_id)
print deleted

The operation of submitting a query, downloading a file, converting it to a `pandas` frame, and deleteing a job will be something will re-use again so let us wrap this procedure into a couple of functions:

In [ ]:
def submit_query(client, query, queue):
    parameters = {'query': query, 'queue': queue}
    job = client.new_job(parameters)
    time.sleep(1)
    run = client.run_job(job.job_id)
    
    return run
    
def get_data(client, run, username, password, wait='10',
             filename='res.csv'):
    job = client.get_job(run.job_id, wait=wait, phase='QUEUED')
    
    if job.phase[0] == 'COMPLETED':
        fileurl = str(job.results[0].reference)
        client.connection.download_file(fileurl, username, password,
                                        file_name=filename)
        data = pd.read_csv(filename)
        success = client.delete_job(job.job_id)
        return data
    else:
        print 'Job in phase %s' % (job.phase[0])
        return job

### Example queries

##### 1. Magnitude histogram

Executing a query and fetching the results can now be done in a couple of lines. A histogram of G magnitudes of TGAS stars can be produced with the following query:

In [ ]:
query = '''
        SELECT FLOOR(phot_g_mean_mag * 10) / 10.0 AS gmag,
               COUNT(FLOOR(phot_g_mean_mag)) AS count
        FROM GDR1.tgas_source
        GROUP BY FLOOR(phot_g_mean_mag * 10)
        '''
run = submit_query(cli, query, queue='long')

In [ ]:
data = get_data(cli, run, username, password)

We can print it out to see we really got what we expected.

In [ ]:
data

And plot it:

In [ ]:
ax = plt.subplot(111)
ax.step(data['gmag'], data['count'])
ax.set_yscale('log')
ax.set_xlabel('gmag')
ax.set_ylabel('count')
plt.show()

##### 2. Density plot

To replicate the TGAS star density diagrams from <a href="https://arxiv.org/abs/1609.04303"> Lindegren et al.</a>, we use the following query. Gaia catalog `source_id` also encodes HEALPix values in nested ordering up to resolution index of 12. To get HEALPix values at lower resolutions we need to divide the `source_id` column with an appropriate factor.

In [ ]:
query = '''
        SELECT hpix, COUNT(hpix) AS number
        FROM
        ( 
            SELECT FLOOR(source_id / (POW(2, 35) * POW(4, 12 - 6))) as hpix
            FROM GDR1.tgas_source
        ) AS hq
        GROUP BY hpix
        '''
run = submit_query(cli, query, queue='long')        

In [ ]:
data = get_data(cli, run, username, password)

In [ ]:
density = np.zeros(12 * (2 ** 6) ** 2)
for hpix, c in zip(data['hpix'], data['number']):
    density[hpix] = c
density /= 0.8393  # area of one HEALPix with NSIDE=64
    
cmap = plt.cm.magma
cmap.set_under('w')

hp.mollview(density, nest=True, title='Source density [deg^2]', cmap=cmap,
            coord='C', norm='log', min=1, max=220)
hp.mollview(density, nest=True, title='', cmap=cmap,
            coord='CG', norm='log', min=1, max=220, cbar=False)
plt.show()

##### 3. Radial velocities from RAVE

Here is a slightly more elaborate example combining the positions of TGAS stars with the radial velocities from RAVE DR5 catalog. It joins two tables that are linked by the `source_id` columns.

In [ ]:
query = '''
        SELECT tgas.l AS l, tgas.b AS b, tgas.parallax AS par,
               rave.HRV AS rv, rave.logg_K AS logg
        FROM GDR1.tgas_source AS tgas, RAVE.RAVE_DR5 AS rave
        WHERE rave.ALGO_CONV = 0 AND rave.logg_K > 0
        AND tgas.source_id = rave.source_id
        LIMIT 50000
        '''
run = submit_query(cli, query, queue='long')

In [ ]:
data = get_data(cli, run, username, password)

In [ ]:
def pscatter(ax, x, y, c='k', size=5, cmap=plt.cm.jet):
    x = np.remainder(x + 360.0, 360.0)
    x[x > 180.0] -= 360.0
    sc = ax.scatter(np.radians(-x), np.radians(y), c=c, s=size, lw=0, cmap=cmap)
    return sc

fig = plt.figure(figsize=(9, 4))
ax = plt.subplot(111, projection='aitoff')
rv = np.clip(data['rv'], -50, 50)
sc = pscatter(ax, data['l'], data['b'], c=rv, size=2)
cbar = plt.colorbar(sc)
cbar.set_label('HRV [km/s]')
ax.set_xticklabels([])
plt.grid()
plt.show()

Or in 3D using the TGAS parallax as a distance estimator. In this example we start a simple HTTP server to host the 3D viewer. Afterwards the server has to be stopped using the `kill()` command.

In [ ]:
color = plt.cm.jet((rv + 50.0) / 100.0)
data['rc'] = color[:, 0]
data['gc'] = color[:, 1]
data['bc'] = color[:, 2]
data.to_csv('threedviewer/res.csv')

In [ ]:
# Galaxy image credit: R. Hurt (SSC), JPL-Caltech, NASA

import subprocess
import webbrowser

server = subprocess.Popen(['python', '-m', 'SimpleHTTPServer', '8889'])
url_viewer = 'http://localhost:8889/threedviewer'
wb = webbrowser.open(url_viewer, new=2)

In [ ]:
server.kill()